In [5]:
import os
import cv2
import matplotlib.pyplot as plt
from IPython.display import clear_output
import shutil
import numpy as np

In [16]:
class BBDatasetGroundtruthTMS:
    
    def __init__(self, image_folder, output_folder, start_index=0):
        """
        'image folder' contains 2 folders : 'helipad' and 'false_positive'
        """
        self.image_folder = image_folder
        self.output_folder = output_folder
        self.start_index = start_index
        
        if not os.path.isdir(os.path.join(output_folder, 'helipad')):
            os.mkdir(os.path.join(output_folder, 'helipad'))
        if not os.path.isdir(os.path.join(output_folder, 'false_positive')):
            os.mkdir(os.path.join(output_folder, 'false_positive'))
    
    def build_target_files(self):
        target_files = []
        for subdir, dirs, files in os.walk(self.image_folder, topdown=True):
            for file in files:
                target_files.append(os.path.join(subdir, file))
        return target_files
    
    def run(self):
        
        target_files = self.build_target_files()
        print(f'{len(target_files)} files loaded!')
        
        l = len(target_files)
        nb_move = 0
        i = self.start_index
        
        while i < l:
            print(i)
            print(f'{l-i} files remaining!')
            
            filepath = target_files[i]
            print(filepath)
            
            image = plt.imread(filepath)

            plt.imshow(image)
            plt.show()

            key = input()

            while key != 'y' and key != 'n' and key != 'p':
                key = input()

            print(key)
            
            if key == 'p':
                i = i-1
                continue
            if key == 'y':
                # yes, move image from input to output
                shutil.copy(filepath,
                            os.path.join(self.output_folder, 
                                        'helipad',
                                        os.path.basename(filepath)))
            elif key == 'n':
                shutil.copy(filepath,
                           os.path.join(self.output_folder, 
                                        'false_positive',
                                        os.path.basename(filepath)))
            
            i = i + 1
            clear_output()

In [17]:
image_folder = "C:\\Users\AISG\\Documents\\Jonas\\Helipad\\Real_World_Detected_Boxes\\model_10_0.0\\sat"
output_folder = "C:\\Users\AISG\\Documents\\Jonas\\Helipad\\Real_World_Detected_Boxes\\model_10_0.0_groundtruth\\"
start_index = 0

bb_dataset_cleaner = BBDatasetGroundtruthTMS(image_folder=image_folder,
                                     output_folder=output_folder,
                                     start_index=start_index)

bb_dataset_cleaner.run()

In [18]:
class BBDatasetCleanerTMS:
    
    def __init__(self, image_folder, check_false_positive=True, start_index=0):
        """
        'image folder' contains 2 folders : 'helipad' and 'false_positive'
        """
        self.image_folder = image_folder
        self.check_false_positive = check_false_positive
        self.start_index = start_index
        
        if check_false_positive:
            self.input_folder = os.path.join(self.image_folder,
                                               'false_positive')
            self.output_folder = os.path.join(self.image_folder,
                                             'helipad')
        else:
            self.input_folder = os.path.join(self.image_folder,
                                               'helipad')
            self.output_folder = os.path.join(self.image_folder,
                                             'false_positive')
    
    def build_target_files(self):
        target_files = []
        for subdir, dirs, files in os.walk(self.input_folder, topdown=True):
            for file in files:
                target_files.append(os.path.join(subdir, file))
        return target_files
    
    def run(self):
        
        target_files = self.build_target_files()
        l = len(target_files)
        nb_move = 0
        i = self.start_index
        
        while i < l:
            print(i)
            print(f'{l-i} files remaining!')
            
            filepath = target_files[i]
            print(filepath)
            
            image = plt.imread(filepath)

            plt.imshow(image)
            plt.show()

            key = input()

            while key != 'y' and key != 'n' and key != 'p':
                key = input()

            print(key)
            
            if key == 'p':
                i = i-1
                continue
            if key == 'y':
                if not os.path.isfile(os.path.join(self.output_folder, os.path.basename(filepath))):                              
                    # yes, move image from input to output
                    shutil.move(filepath,
                               self.output_folder, os.path.basename(filepath))  
                    print('Image moved')
                else:
                    os.remove(os.path.join(self.input_folder, os.path.basename(filepath)))
                    print('Image deleted')
                                      
                nb_move += 1
            
            i = i + 1
            clear_output()

In [19]:
image_folder = "C:\\Users\AISG\\Documents\\Jonas\\Helipad\\Real_World_Detected_Boxes\\model_10_0.0_groundtruth\\"
start_index = 0

bb_dataset_cleaner = BBDatasetCleanerTMS(image_folder=image_folder,
                                         check_false_positive=True,
                                         start_index=start_index)

bb_dataset_cleaner.run()

In [20]:
np.random.seed(42)

class BBDatasetGroundtruthTMSSplitTestTrain:
    
    def __init__(self, image_folder, output_folder, test_size=0.2):
        
        self.image_folder = image_folder
        self.output_folder = output_folder
        self.test_size = test_size
    
    def build_target_files(self, classe='helipad'):
        target_files = []
        for subdir, dirs, files in os.walk(os.path.join(self.image_folder, classe), topdown=True):
            for file in files:
                target_files.append(os.path.join(subdir, file))
        return target_files
    
    def run(self):
        
        helipad_path = self.build_target_files(classe='helipad')
        false_positive_path = self.build_target_files(classe='false_positive')
        
        l_heli = len(helipad_path)
        l_fp = len(false_positive_path)
        
        nb_heli_test = int(l_heli*(self.test_size))
        index_heli_test = np.random.choice(l_heli, nb_heli_test)
        index_heli_train = [i for i in range(l_heli) if i not in index_heli_test]
        
        nb_fp_test = int(l_fp*(self.test_size))
        index_fp_test = np.random.choice(l_fp, nb_fp_test)
        index_fp_train = [i for i in range(l_fp) if i not in index_fp_test]
        
        helipad_train_path = np.array(helipad_path)[index_heli_train]
        helipad_test_path = np.array(helipad_path)[index_heli_test]
        
        fp_train_path = np.array(false_positive_path)[index_fp_train]
        fp_test_path = np.array(false_positive_path)[index_fp_test]
        
        # copy file to train/helipad train/false_positive
        # copy file to test/helipad test/false_positive
        
        if not os.path.isdir(os.path.join(output_folder, 'train')):
            os.mkdir(os.path.join(output_folder, 'train'))
        if not os.path.isdir(os.path.join(output_folder, 'test')):
            os.mkdir(os.path.join(output_folder, 'test'))
        if not os.path.isdir(os.path.join(output_folder, 'train', 'helipad')):
            os.mkdir(os.path.join(output_folder, 'train', 'helipad'))
        if not os.path.isdir(os.path.join(output_folder, 'train', 'false_positive')):
            os.mkdir(os.path.join(output_folder, 'train', 'false_positive'))
        if not os.path.isdir(os.path.join(output_folder, 'test', 'helipad')):
            os.mkdir(os.path.join(output_folder, 'test', 'helipad'))
        if not os.path.isdir(os.path.join(output_folder, 'test', 'false_positive')):
            os.mkdir(os.path.join(output_folder, 'test', 'false_positive'))
        
        for path in helipad_train_path:
            shutil.copy(path, os.path.join(output_folder, 'train', 'helipad', os.path.basename(path)))
        for path in helipad_test_path:
            shutil.copy(path, os.path.join(output_folder, 'test', 'helipad', os.path.basename(path)))
        for path in fp_train_path:
            shutil.copy(path, os.path.join(output_folder, 'train', 'false_positive', os.path.basename(path)))
        for path in fp_test_path:
            shutil.copy(path, os.path.join(output_folder, 'test', 'false_positive', os.path.basename(path)))
        
        print('Done')

In [21]:
image_folder = "C:\\Users\AISG\\Documents\\Jonas\\Helipad\\Real_World_Detected_Boxes\\model_10_0.0_groundtruth\\"
output_folder = "C:\\Users\AISG\\Documents\\Jonas\\Helipad\\Real_World_Detected_Boxes\\model_10_0.0_groundtruth_split\\"
test_size = 0.2

bb_dataset_groundtruth_tms_split_test_train = BBDatasetGroundtruthTMSSplitTestTrain(image_folder=image_folder,
                                                                                   output_folder=output_folder,
                                                                                   test_size=test_size)

bb_dataset_groundtruth_tms_split_test_train.run()

Done


In [1]:
from bb_training_manager import BBTrainingManager

image_folder = "C:\\Users\AISG\\Documents\\Jonas\\Helipad\\Real_World_Detected_Boxes\\model_10_0.0_groundtruth_split\\"
model_filename = "bb_model_manilla_adam_64.h5"

bbtraining_manager = BBTrainingManager(image_folder)

bbtraining_manager.run()

bbtraining_manager.evaluate()

bbtraining_manager.save(filename=model_filename)

bbtraining_manager.plot()

Using TensorFlow backend.
C:\Users\AISG\Anaconda3\envs\tf-gpu_1.13\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\AISG\Anaconda3\envs\tf-gpu_1.13\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\AISG\Anaconda3\envs\tf-gpu_1.13\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\AISG\Anaconda3\envs\tf-

Found 464 images belonging to 2 classes.
Found 102 images belonging to 2 classes.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Epoch 1/500
1/1 [==============================] - 3s 3s/step - loss: 0.9935 - acc: 0.1164 - val_loss: 0.9100 - val_acc: 0.8824
Epoch 2/500
1/1 [==============================] - 0s 40ms/step - loss: 0.8990 - acc: 0.8836 - val_loss: 0.5282 - val_acc: 0.8824
Epoch 3/500
1/1 [==============================] - 0s 159ms/step - loss: 0.5196 - acc: 0.8836 - val_loss: 0.4708 - val_acc: 0.8824
Epoch 4/500
1/1 [==============================] - 0s 197ms/step - loss: 0.4667 - acc: 0.8836 - val_loss: 0.4136 - val_acc: 0.8824
Epoch 5/500
1/1 [==============================] - 0s 239ms/step - loss: 0.4029 - acc: 0.8836 - val_loss: 0.4523 - val_acc: 0.8824
Epoch 6/500
1/1 [==============================] - 0s 165ms/step - loss: 0.4395 - acc: 0.8836 - val_loss: 0.3888 - val_acc: 0.8824
Epoch 7/500
1/1 

1/1 [==============================] - 0s 251ms/step - loss: 0.0792 - acc: 0.9677 - val_loss: 0.2693 - val_acc: 0.9118
Epoch 59/500
1/1 [==============================] - 0s 155ms/step - loss: 0.0709 - acc: 0.9741 - val_loss: 0.2750 - val_acc: 0.9118
Epoch 60/500
1/1 [==============================] - 0s 228ms/step - loss: 0.0668 - acc: 0.9763 - val_loss: 0.2371 - val_acc: 0.9216
Epoch 61/500
1/1 [==============================] - 0s 180ms/step - loss: 0.0623 - acc: 0.9763 - val_loss: 0.2466 - val_acc: 0.9314
Epoch 62/500
1/1 [==============================] - 0s 199ms/step - loss: 0.0540 - acc: 0.9828 - val_loss: 0.2966 - val_acc: 0.9216
Epoch 63/500
1/1 [==============================] - 0s 234ms/step - loss: 0.0543 - acc: 0.9784 - val_loss: 0.2510 - val_acc: 0.9314
Epoch 64/500
1/1 [==============================] - 0s 172ms/step - loss: 0.0451 - acc: 0.9914 - val_loss: 0.2488 - val_acc: 0.9216
Epoch 65/500
1/1 [==============================] - 0s 237ms/step - loss: 0.0422 - acc: 0

Epoch 120/500
1/1 [==============================] - 0s 195ms/step - loss: 6.7035e-05 - acc: 1.0000 - val_loss: 0.6004 - val_acc: 0.9314
Epoch 121/500
1/1 [==============================] - 0s 242ms/step - loss: 6.4404e-05 - acc: 1.0000 - val_loss: 0.6006 - val_acc: 0.9314
Epoch 122/500
1/1 [==============================] - 0s 165ms/step - loss: 6.1712e-05 - acc: 1.0000 - val_loss: 0.6017 - val_acc: 0.9314
Epoch 123/500
1/1 [==============================] - 0s 244ms/step - loss: 5.9003e-05 - acc: 1.0000 - val_loss: 0.6033 - val_acc: 0.9314
Epoch 124/500
1/1 [==============================] - 0s 164ms/step - loss: 5.6416e-05 - acc: 1.0000 - val_loss: 0.6053 - val_acc: 0.9314
Epoch 125/500
1/1 [==============================] - 0s 238ms/step - loss: 5.4048e-05 - acc: 1.0000 - val_loss: 0.6073 - val_acc: 0.9314
Epoch 126/500
1/1 [==============================] - 0s 172ms/step - loss: 5.1936e-05 - acc: 1.0000 - val_loss: 0.6091 - val_acc: 0.9314
Epoch 127/500
1/1 [======================

Epoch 180/500
1/1 [==============================] - 0s 202ms/step - loss: 1.5700e-05 - acc: 1.0000 - val_loss: 0.6163 - val_acc: 0.9314
Epoch 181/500
1/1 [==============================] - 0s 233ms/step - loss: 1.5466e-05 - acc: 1.0000 - val_loss: 0.6164 - val_acc: 0.9314
Epoch 182/500
1/1 [==============================] - 0s 175ms/step - loss: 1.5241e-05 - acc: 1.0000 - val_loss: 0.6163 - val_acc: 0.9314
Epoch 183/500
1/1 [==============================] - 0s 201ms/step - loss: 1.5017e-05 - acc: 1.0000 - val_loss: 0.6163 - val_acc: 0.9314
Epoch 184/500
1/1 [==============================] - 0s 237ms/step - loss: 1.4800e-05 - acc: 1.0000 - val_loss: 0.6163 - val_acc: 0.9314
Epoch 185/500
1/1 [==============================] - 0s 172ms/step - loss: 1.4585e-05 - acc: 1.0000 - val_loss: 0.6164 - val_acc: 0.9314
Epoch 186/500
1/1 [==============================] - 0s 226ms/step - loss: 1.4378e-05 - acc: 1.0000 - val_loss: 0.6164 - val_acc: 0.9314
Epoch 187/500
1/1 [======================

Epoch 240/500
1/1 [==============================] - 0s 162ms/step - loss: 7.4786e-06 - acc: 1.0000 - val_loss: 0.6226 - val_acc: 0.9314
Epoch 241/500
1/1 [==============================] - 0s 255ms/step - loss: 7.4000e-06 - acc: 1.0000 - val_loss: 0.6227 - val_acc: 0.9314
Epoch 242/500
1/1 [==============================] - 0s 150ms/step - loss: 7.3244e-06 - acc: 1.0000 - val_loss: 0.6229 - val_acc: 0.9314
Epoch 243/500
1/1 [==============================] - 0s 204ms/step - loss: 7.2498e-06 - acc: 1.0000 - val_loss: 0.6230 - val_acc: 0.9314
Epoch 244/500
1/1 [==============================] - 0s 224ms/step - loss: 7.1767e-06 - acc: 1.0000 - val_loss: 0.6231 - val_acc: 0.9314
Epoch 245/500
1/1 [==============================] - 0s 184ms/step - loss: 7.1044e-06 - acc: 1.0000 - val_loss: 0.6232 - val_acc: 0.9314
Epoch 246/500
1/1 [==============================] - 0s 235ms/step - loss: 7.0321e-06 - acc: 1.0000 - val_loss: 0.6233 - val_acc: 0.9314
Epoch 247/500
1/1 [======================

In [3]:
from bb_predict import BBPredict

image_folder = "C:\\Users\\AISG\\Documents\\Jonas\\Real_World_Dataset_TMS\\sat\\"
meta_folder = "C:\\Users\\AISG\\Documents\\Jonas\\Real_World_Dataset_TMS_meta\\sat\\"
model_number = 10
model_path = "./checkpoint_manilla/weights.81-0.01-1.00-0.32-0.94.h5"
tms = True
extract_bounding_boxes = True

index_path = "../helipad_path_over_0_m10.txt"

bbpredict = BBPredict(image_folder=image_folder,
                      meta_folder=meta_folder,
                      model_number=model_number,
                      model_path=model_path,
                      tms=tms,
                      extract_bounding_boxes=extract_bounding_boxes,
                      index_path=index_path)

bbpredict.run()

100%|███████████████████████████████████████████████████████████████████████████████| 568/568 [00:01<00:00, 345.01it/s]


In [8]:
import os
os.getcwd()
os.chdir('..')
os.getcwd()

'C:\\Users\\AISG\\Documents\\Jonas\\helipad_detection\\src'

In [9]:
import sys
sys.path.append('..')
sys.path.append('detection')
from detection.build_placemarks import BuildPlacemarks

meta_folder = "C:\\Users\\AISG\\Documents\\Jonas\\Real_World_Dataset_TMS_meta\\sat\\"
model_number = 10
threshold = 0.7

index_path = "helipad_path_over_0_m10.txt"
# index_path = None

knn = True
model_name = "cnn_validation"
model_validation_threshold = 0.7

build_placemarks = BuildPlacemarks(meta_folder,
                                   model_number,
                                   threshold,
                                   knn=knn,
                                   model_name=model_name,
                                   model_validation_threshold=model_validation_threshold,
                                   index_path=index_path)

build_placemarks.run()

100%|███████████████████████████████████████████████████████████████████████████████| 568/568 [00:01<00:00, 369.57it/s]
